In [2]:
import pandas as pd
pd.set_option('display.max_columns', 500)

In [32]:
def find_warsaw_district(input_string) -> str:
    """
    Identifies and returns a Warsaw district name from a given input string.

    The function splits the input string by commas and checks each part against a list of Warsaw district names. 
    If one of the parts matches a district name, that district name is returned. If there are no matches, 
    the function returns None.

    Parameters:
    - input_string (str): The string to be analyzed, which can contain multiple comma-separated values.

    Returns:
    - str or None: The name of a Warsaw district if found; otherwise, None.
    """
    # List of Warsaw districts
    warsaw_districts = [
        "Bemowo", "Białołęka", "Bielany", "Mokotów", "Ochota",
        "Praga-Południe", "Praga-Północ", "Rembertów", "Śródmieście",
        "Targówek", "Ursus", "Ursynów", "Wawer", "Wesoła",
        "Wilanów", "Włochy", "Wola", "Żoliborz"
    ]

    # Split the input string by commas
    parts = input_string.split(',')

    # Check each part for a match with Warsaw districts
    for part in parts:
        if part.strip() in warsaw_districts:
            return part.strip()  # Return the matching district name

    return None

In [34]:
def extract_ad_dates(row):
    ad_info = row['announcement_date']
    
    update_date_part = [line for line in ad_info.split('\\n') if 'Aktualizacja:' in line]
    last_update = update_date_part[0].replace("('Aktualizacja: ", '').strip() if update_date_part else None
    
    added_date_part = [line for line in ad_info.split('\\n') if 'Dodano:' in line]
    ad_added = added_date_part[0].replace('Dodano: ', '').strip() if added_date_part else None
    
    return pd.Series([last_update, ad_added])

In [36]:
def process_data(data_draw):
    
    df = data_draw.copy()
    df['district'] = df['location'].apply(lambda x: find_warsaw_district(str(x)))
    df = df[~df.district.isna()]
    
    df[['last_update', 'added_dt']] = df.apply(extract_ad_dates, axis=1)
    df.drop(['announcement_date'], axis=1, inplace=True)
    df['expired'] = 0
    df['expired_date'] = None
    to_order = ['added_dt', 'last_update', 'link', 'expired', 'expired_date']
    columns_order = to_order + [col for col in df.columns if col not in to_order]
    df = df[columns_order]
    
    return df

In [38]:
df = pd.read_csv('data_raw\otodom_scraped_data\otodom_2025_01_05.csv')

In [40]:
df = process_data(df)

In [4]:
import pandas as pd

In [18]:
path = f'data_processed/main.csv'
main = pd.read_csv(path)

In [22]:
main[main.expired.eq(1)]

,added_dt,last_update,link,expired,expired_date,title,rent_price,area_room_num,floor,ogrzewanie,flat_condition,available_from,deposit,advertiser_type,additional_information,location,latitude,longitude,approximate_coordinates,year_of_construction,elevator,building_type,security,equipment,utilities,safeguards,adv_description,district
0,22.12.2024,22.12.2024,https://www.otodom.pl/pl/oferta/piekne-mieszka...,1,2025_01_04,Piękne mieszkanie na warszawskich Skoroszach,3 000 zł\n/miesiąc\n+ Czynsz 600 zł,45m²\nWynajmę również studentom\n2 pokoje,3/6,inne,do zamieszkania,NaN,3 000 zł,prywatny,balkon\ngaraż/miejsce parkingowe,"ul. Skoroszewska, Skorosze, Ursus, Warszawa, m...",52.189750,20.899275,False,2013.0,tak,apartamentowiec,teren zamknięty\nmonitoring / ochrona,zmywarka\nlodówka\nmeble\npiekarnik\nkuchenka\...,NaN,drzwi / okna antywłamaniowe\ndomofon / wideofon,Wynajmę mieszkanie 45 metrów na warszawskim Ur...,Ursus
1,15.12.2024,21.12.2024,https://www.otodom.pl/pl/oferta/zarezerwowane-...,1,2025_01_08,[ZAREZERWOWANE] przy Metrze Daszyńskiego,3 950 zł\n/miesiąc\n+ Czynsz 690 zł,42m²\n2 pokoje,2/8,miejskie,do zamieszkania,NaN,8 000 zł,prywatny,balkon\ngaraż/miejsce parkingowe,"ul. Karolkowa, Czyste, Wola, Warszawa, mazowie...",52.226621,20.978226,False,2020.0,tak,apartamentowiec,teren zamknięty\nmonitoring / ochrona,zmywarka\nlodówka\nmeble\npiekarnik\nkuchenka\...,internet,drzwi / okna antywłamaniowe\ndomofon / wideofon,English version below.\n\nMIESZKANIE\nNowoczes...,Wola
2,18.12.2024,18.12.2024,https://www.otodom.pl/pl/oferta/nowe-2-pokojow...,1,2025_01_02,Nowe 2-pokojowe wysoki standard| Saska Kępa|Garaż,4 700 zł\n/miesiąc\n+ Czynsz 750 zł,45m²\n2 pokoje\ntylko dla niepalących,2/4,miejskie,do zamieszkania,NaN,NaN,prywatny,balkon\ngaraż/miejsce parkingowe\ntylko dla ni...,"Osiedle Lizbońska, Saska Kępa, Praga-Południe,...",52.228044,21.063960,False,2024.0,tak,apartamentowiec,monitoring / ochrona,zmywarka\nlodówka\nmeble\npiekarnik\nkuchenka\...,NaN,domofon / wideofon,Do wynajęcia NOWE 2-pokojowe mieszkanie o powi...,Praga-Południe
3,14.12.2023,22.12.2024,https://www.otodom.pl/pl/oferta/3-pok-70m2-pow...,1,2025_01_09,"3 pok, 70m2, Powstańców Śl. Metro Bemowo, Garaż",4 200 zł\n/miesiąc\n+ Czynsz 911 zł,70m²\n3 pokoje\ntylko dla niepalących,3/7,miejskie,do zamieszkania,NaN,6 500 zł,prywatny,balkon\ntaras\ngaraż/miejsce parkingowe\npiwni...,"ul. Powstańców Śląskich 89c, Ewen Shiraz, Górc...",52.243598,20.910004,False,2008.0,tak,apartamentowiec,teren zamknięty\nmonitoring / ochrona,zmywarka\nlodówka\nmeble\npiekarnik\nkuchenka\...,NaN,domofon / wideofon,BEZPOŚREDNIO OD WŁAŚCICIELA - dostępne - Najem...,Bemowo
5,28.11.2024,22.12.2024,https://www.otodom.pl/pl/oferta/2-pokojowe-mie...,1,2024_12_29,2-pokojowe mieszkanie w klimatycznej kamienicy,3 500 zł\n/miesiąc\n+ Czynsz 700 zł,48m²\n2 pokoje\ntylko dla niepalących,1/4,miejskie,do zamieszkania,NaN,7 000 zł,prywatny,balkon\ntaras\ntylko dla niepalących,"Nowa Praga, Praga-Północ, Warszawa, mazowieckie",52.263300,21.028320,True,2019.0,tak,kamienica,teren zamknięty,zmywarka\nlodówka\nmeble\npiekarnik\nkuchenka\...,internet,drzwi / okna antywłamaniowe\ndomofon / wideofon,Do wynajęcia 2 pokojowe mieszkanie wraz z duży...,Praga-Północ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8804,12.01.2025,12.01.2025,https://www.otodom.pl/pl/oferta/3-pokoje-nowe-...,1,2025_01_14,3 pokoje nowe budownictwo ochrona garaż x 2 metro,4 300 zł\n/miesiąc\n+ Czynsz 1 144 zł,54.3m²\n3 pokoje,5/6,miejskie,do zamieszkania,NaN,5 000 zł,prywatny,balkon\ngaraż/miejsce parkingowe,"ul. Józefa Sowińskiego 53b, Ulrychów, Wola, Wa...",52.231284,20.933721,False,2013.0,tak,apartamentowiec,teren zamknięty\nmonitoring / ochrona,zmywarka\nlodówka\nmeble\npiekarnik\nkuchenka\...,telewizja kablowa\ninternet,domofon / wideofon,Wynajmę 3 pokojowe mieszkanie o powierzchni 54...,Wola
8831,NaN,NaN,https://www.otodom.pl/pl/oferta/wynajme-kawale...,1,2025_01_14,Wynajmę

In [8]:
sorted([date for date in list(main.expired_date.unique()) if isinstance(date, str)])

['2024_12_29',
 '2025_01_02',
 '2025_01_03',
 '2025_01_04',
 '2025_01_06',
 '2025_01_07',
 '2025_01_08',
 '2025_01_09',
 '2025_01_10',
 '2025_01_11',
 '2025_01_12',
 '2025_01_13',
 '2025_01_15']

In [12]:
main['expired_date'] = main['expired_date'].replace('2025_01_15', '2025_01_14')

In [14]:
sorted([date for date in list(main.expired_date.unique()) if isinstance(date, str)])

['2024_12_29',
 '2025_01_02',
 '2025_01_03',
 '2025_01_04',
 '2025_01_06',
 '2025_01_07',
 '2025_01_08',
 '2025_01_09',
 '2025_01_10',
 '2025_01_11',
 '2025_01_12',
 '2025_01_13',
 '2025_01_14']

In [26]:
main = pd.concat([main, df], ignore_index=True)

NameError: name 'df' is not defined

In [16]:
main.to_csv(path,
          encoding='utf-8',
          index=False)

### ====